In [42]:
import sys
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd

import tskit

In [4]:
treefile = "../param_grid/post_21000_2022-12-05/run_2022-12-05_000018/sim_1876298096991.trees"
outbase = ".".join(treefile.split(".")[:-1])

ts = tskit.load(treefile)
params = ts.metadata['SLiM']['user_metadata']

indiv_times = ts.individual_times
indiv_pops = ts.individual_populations
indiv_locs = ts.individual_locations

In [8]:
modern = np.logical_and(
        indiv_times < 100,
        indiv_pops == 1
)
target_indivs = [
    np.where(np.logical_and(
        modern,
        indiv_locs[:, 1] == np.max(indiv_locs[modern, 1]),
    ))[0][0],
    np.where(np.logical_and(
        modern,
        indiv_locs[:, 1] == np.min(indiv_locs[modern, 1]),
    ))[0][0],
]


In [10]:
node_anc = ts.sample_count_stat(
        [ts.individual(n).nodes for n in target_indivs],
        lambda x: x/2, # 2 for diploidy
        2,
        polarised=True,
        strict=False,
        mode='node'
)

In [13]:
indiv_anc = np.zeros((ts.num_individuals, len(target_indivs)))
for n in ts.nodes():
    if n.individual >= 0:
        indiv_anc[n.individual] += node_anc[n.id]

In [18]:
tts = list(set(indiv_times))
tts.sort()

In [47]:
all_times = []
all_locs1 = []
all_locs2 = []
all_anc_props = []
all_inds = []
for k, ind in enumerate(target_indivs):
    j = 0
    for target_time in tts:
        anc_indivs = np.where(np.logical_and(
            indiv_times == target_time,
            indiv_pops == 1 # exclude dummy indiv in pop 2
        ))[0]
        assert len(anc_indivs) > 0
        anc_props = indiv_anc[anc_indivs, k]
        locs1 = indiv_locs[anc_indivs, 0]
        locs2 = indiv_locs[anc_indivs, 1]
        all_times.extend(np.repeat(target_time, len(locs1)))
        all_inds.extend(np.repeat(ind, len(locs1)))
        all_locs1.extend(locs1)
        all_locs2.extend(locs2)
        all_anc_props.extend(anc_props)
        j += 1

In [50]:
output = pd.DataFrame({'ind': all_inds, 'anc_prop': all_anc_props, 'loc1': all_locs1, 'loc2': all_locs2, 'time': all_times})
output.to_csv("data_for_ancestry_plot.csv", index = False)

In [61]:
print(indiv_locs[:,:])

[[108.93629419 152.64246186   0.        ]
 [109.15236551 151.98279078   0.        ]
 [108.96681187 152.27507954   0.        ]
 ...
 [134.64343052  66.73142764   0.        ]
 [134.2587588   68.70479536   0.        ]
 [134.2587588   68.70479536   0.        ]]


In [65]:
# Write out locations over time of all individuals
indiv_output = pd.DataFrame({'time': indiv_times, 'loc1': indiv_locs[:,0], 'loc2': indiv_locs[:,1]})
print(indiv_output)
indiv_output.to_csv("data_for_all_inds.csv", index = False)

          time        loc1        loc2
0          0.0  108.936294  152.642462
1          0.0  109.152366  151.982791
2          0.0  108.966812  152.275080
3          0.0  108.936294  152.642462
4          0.0  108.936294  152.642462
...        ...         ...         ...
214553  2250.0  136.606512   67.608720
214554  2250.0  134.643431   66.731428
214555  2250.0  134.643431   66.731428
214556  2250.0  134.258759   68.704795
214557  2250.0  134.258759   68.704795

[214558 rows x 3 columns]
